# Docker compose

Это надстройка над докером, которая позволяет запускать множество контейнеров одновременно и маршрутизировать потоки данных между ними. (написанна на Python)
Для каждого проекта (кластера контейнеров) Docker создаёт свою сеть, где контейнеры могут обращаться друг к другу по именам, которые мы укажем в docker-compose.yml.

Убедимся, что при установке Docker сразу подтягивается Docker-compose

In [3]:
! pip install docker

Defaulting to user installation because normal site-packages is not writeable


Основную информацию и базовые команды получаем стандартным флагом ```--help```

In [6]:
! docker compose --help


Usage:  docker compose [OPTIONS] COMMAND

Define and run multi-container applications with Docker

Options:
      --all-resources              Include all resources, even those not
                                   used by services
      --ansi string                Control when to print ANSI control
                                   characters ("never"|"always"|"auto")
                                   (default "auto")
      --compatibility              Run compose in backward compatibility mode
      --dry-run                    Execute command in dry run mode
      --env-file stringArray       Specify an alternate environment file
  -f, --file stringArray           Compose configuration files
      --parallel int               Control max parallelism, -1 for
                                   unlimited (default -1)
      --profile stringArray        Specify a profile to enable
      --progress string            Set type of progress output (auto,
                                 

Пример: 

In [9]:
! cat docker-compose.yml

158.16s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


services:
  redis:
    image: redis
  rocketcounter:
    image: rotorocloud/rocket-counter
    ports:
    - 8085:5000
version: '3.0'

In [11]:
! docker-compose up -d

217.92s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Creating network "docker_learning_course_default" with the default driver
Creating docker_learning_course_redis_1 ... 
Creating docker_learning_course_rocketcounter_1 ... 
ting docker_learning_course_redis_1         ... done

In [13]:
! docker-compose ps 

241.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


         Name                   Command           State           Ports         
--------------------------------------------------------------------------------
docker_learning_course   docker-entrypoint.sh     Up      6379/tcp              
_redis_1                 redis ...                                              
docker_learning_course   flask run                Up      0.0.0.0:8085-         
_rocketcounter_1                                          >5000/tcp,:::8085-    
                                                          >5000/tcp             


Сейчас на [localhost:8085](http://127.0.0.1:8085/) доступен 1 экземпляр тестового приложения. Логи приложения можно посмотреть командой:

In [16]:
! docker-compose logs rocketcounter

516.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Attaching to docker_learning_course_rocketcounter_1
rocketcounter_1  |  * Serving Flask app 'app.py'
rocketcounter_1  |  * Debug mode: off
rocketcounter_1  | WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
rocketcounter_1  |  * Running on all addresses (0.0.0.0)
rocketcounter_1  |  * Running on http://127.0.0.1:5000
rocketcounter_1  |  * Running on http://172.20.0.2:5000
rocketcounter_1  | Press CTRL+C to quit
rocketcounter_1  | 172.20.0.1 - - [05/Aug/2024 16:02:28] "GET / HTTP/1.1" 200 -
rocketcounter_1  | 172.20.0.1 - - [05/Aug/2024 16:02:28] "GET /favicon.ico HTTP/1.1" 404 -
rocketcounter_1  | 172.20.0.1 - - [05/Aug/2024 16:05:13] "GET /getimage HTTP/1.1" 200 -
rocketcounter_1  | 172.20.0.1 - - [05/Aug/2024 16:05:13] "GET /static/average.png HTTP/1.1" 200 -
rocketcounter_1  | 172.20.0.1 - - [05/Aug/2024 16:05:14] "GET /getimage HTTP/1.1" 200 -
rocketcounter_1  | 172.20.0.1 - - [05/Aug/2024 16:05:14] "GET /static/

Остановим и удалим старые контейнеры и запустим сразу 2 экземпляра rocketcounter, слегка изменив docker-compose.yml

In [19]:
! docker-compose down

786.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Stopping docker_learning_course_redis_1         ... 
Stopping docker_learning_course_rocketcounter_2 ... 
ping docker_learning_course_rocketcounter_2 ... doneRemoving docker_learning_course_redis_1         ... 
Removing docker_learning_course_rocketcounter_1 ... 
Removing docker_learning_course_rocketcounter_2 ... 
Removing network docker_learning_course_default done


In [21]:
! cat docker-compose.yml

1017.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


services:
  redis:
    image: redis
  rocketcounter:
    image: rotorocloud/rocket-counter
    ports:
    - 8085-8087:5000
version: '3.0'

In [20]:
! docker-compose up -d --scale rocketcounter=2
! docker-compose ps 

993.55s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Creating network "docker_learning_course_default" with the default driver
Creating docker_learning_course_rocketcounter_1 ... 
Creating docker_learning_course_rocketcounter_2 ... 
Creating docker_learning_course_redis_1         ... 
ting docker_learning_course_redis_1         ... done

1003.74s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


         Name                   Command           State           Ports         
--------------------------------------------------------------------------------
docker_learning_course   docker-entrypoint.sh     Up      6379/tcp              
_redis_1                 redis ...                                              
docker_learning_course   flask run                Up      0.0.0.0:8086-         
_rocketcounter_1                                          >5000/tcp,:::8086-    
                                                          >5000/tcp             
docker_learning_course   flask run                Up      0.0.0.0:8085-         
_rocketcounter_2                                          >5000/tcp,:::8085-    
                                                          >5000/tcp             


Отлично, теперь есть 2 экземпляра приложения, первое развернуто на [localhost:8085](http://127.0.0.1:8085/), второе - [localhost:8086](http://127.0.0.1:8086/). Они оба общаются с redis, и счетчик увеличивается без ошибок.

После экспериментов удаляем все контейнеры и останавливаем среду.

In [22]:
! docker-compose down

1183.83s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Stopping docker_learning_course_rocketcounter_2 ... 
Stopping docker_learning_course_redis_1         ... 
Stopping docker_learning_course_rocketcounter_1 ... 
ping docker_learning_course_rocketcounter_2 ... doneRemoving docker_learning_course_rocketcounter_2 ... 
Removing docker_learning_course_redis_1         ... 
Removing docker_learning_course_rocketcounter_1 ... 
Removing network docker_learning_course_default done
